In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stat
import statsmodels.api as sm

from pandas_datareader import data
from matplotlib.ticker import FuncFormatter 
from scipy.stats import norm
%matplotlib inline
# %matplotlib widget

In [3]:
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

## Load Data

In [4]:
start_date = '2020-01-01'
end_date = '2020-12-31'
end_date = today

In [5]:
ticker = ['BTC-USD']
panel_data = data.DataReader(ticker, 'yahoo', start_date, end_date)
#get the close price
main_df = panel_data['Close']
main_df.head()

Symbols,BTC-USD
Date,
2019-12-31,7193.599121
2020-01-01,7200.174316
2020-01-02,6985.470215
2020-01-03,7344.884277
2020-01-04,7410.656738


In [6]:
tickers = ['ETH-USD','HEX-USD', 'BNB-USD','XRP-USD','SOL1-USD','DOT1-USD','DOGE-USD','UNI3-USD','AVAX-USD','LUNA1-USD','LINK-USD','ALGO-USD','LTC-USD','BCH-USD','ATOM1-USD','MATIC-USD','ICP1-USD','XLM-USD','FIL-USD','TRX-USD','ETC-USD']
# tickers = ['ETH-USD']

for ticker in tickers:
    try:
        panel_data = data.DataReader(ticker, 'yahoo', start_date, end_date)
        panel_data_close = panel_data['Close']
        main_df = pd.concat([main_df, panel_data_close.rename(ticker)], axis=1)
    except:
        pass

In [7]:
main_df

,BTC-USD,ETH-USD,BNB-USD,XRP-USD,DOGE-USD,UNI3-USD,LINK-USD,LTC-USD,BCH-USD,ICP1-USD,XLM-USD,TRX-USD,ETC-USD
Date,,,,,,,,,,,,,
2019-12-31,7193.599121,129.610855,13.727642,0.192894,0.002028,NaN,1.769471,41.340073,204.623169,NaN,0.045224,0.013306,4.496955
2020-01-01,7200.174316,130.802002,13.689083,0.192667,0.002033,NaN,1.806640,42.017719,204.397537,NaN,0.045451,0.013250,4.502792
2020-01-02,6985.470215,127.410179,13.027011,0.188043,0.002009,NaN,1.741144,39.823013,195.698563,NaN,0.044112,0.012870,4.255888
2020-01-03,7344.884277,134.171707,13.660452,0.193521,0.002145,NaN,1.809708,42.415573,222.412979,NaN,0.045234,0.013435,4.576237
2020-01-04,7410.656738,135.069366,13.891512,0.194355,0.002241,NaN,1.840133,43.326607,226.018692,NaN,0.046272,0.013521,4.726842
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-04,51514.812500,3518.518555,442.706146,1.085503,0.252016,25.664791,27.242628,173.891083,594.042542,54.429611,0.319084,0.096183,54.408985
2021-10-05,55361.449219,3580.562012,435.401367,1.077656,0.255134,25.287689,26.837267,178.705673,618.740845,52.813328,0.354442,0.095521,54.899742
2021-10-06,53805.984375,3587.974854,438.761749,1.068026,0.242923,25.535185,26.834036,178.771729,612.156738,50.995556,0.344107,0.095514,54.310539


## Find Cryptos that are inversely correlated / not correlated

In [8]:
corr_matrix = main_df.corr()

In [11]:
corr_matrix

,BTC-USD,ETH-USD,BNB-USD,XRP-USD,DOGE-USD,UNI3-USD,LINK-USD,LTC-USD,BCH-USD,ICP1-USD,XLM-USD,TRX-USD,ETC-USD
BTC-USD,1.000000,0.886582,0.865273,0.803668,0.680599,0.925660,0.932369,0.944445,0.827588,0.062579,0.944348,0.867677,0.635634
ETH-USD,0.886582,1.000000,0.944534,0.904273,0.887853,0.814087,0.877472,0.871184,0.856561,0.177361,0.856998,0.911577,0.890428
BNB-USD,0.865273,0.944534,1.000000,0.946479,0.904184,0.964058,0.867971,0.886002,0.892939,0.210663,0.865691,0.962427,0.866393
XRP-USD,0.803668,0.904273,0.946479,1.000000,0.882840,0.920550,0.851711,0.859611,0.884406,0.429913,0.846159,0.959646,0.842565
DOGE-USD,0.680599,0.887853,0.904184,0.882840,1.000000,0.745021,0.753549,0.772738,0.863319,0.811101,0.742205,0.848733,0.965338
UNI3-USD,0.925660,0.814087,0.964058,0.920550,0.745021,1.000000,0.938660,0.940303,0.880886,0.462157,0.928042,0.918826,0.637369
LINK-USD,0.932369,0.877472,0.867971,0.851711,0.753549,0.938660,1.000000,0.950858,0.886744,0.590771,0.959103,0.897318,0.691853
LTC-USD,0.944445,0.871184,0.886002,0.859611,0.772738,0.940303,0.950858,1.000000,0.949805,0.630978,0.974565,0.900590,0.711569
BCH-USD,0.827588,0.856561,0.892939,0.884406,0.863319,0.880886,0.886744,0.949805,1.000000,0.752062,0.909248,0.895644,0.815423
ICP1-USD,0.062579,0.177361,0.210663,0.429913,0.811101,0.462157,0.590771,0.630978,0.752062,1.000000,0.857824,0.222901,0.730340


In [12]:
corr_matrix[corr_matrix < 0.8]

,BTC-USD,ETH-USD,BNB-USD,XRP-USD,DOGE-USD,UNI3-USD,LINK-USD,LTC-USD,BCH-USD,ICP1-USD,XLM-USD,TRX-USD,ETC-USD
BTC-USD,NaN,NaN,NaN,NaN,0.680599,NaN,NaN,NaN,NaN,0.062579,NaN,NaN,0.635634
ETH-USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.177361,NaN,NaN,NaN
BNB-USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.210663,NaN,NaN,NaN
XRP-USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.429913,NaN,NaN,NaN
DOGE-USD,0.680599,NaN,NaN,NaN,NaN,0.745021,0.753549,0.772738,NaN,NaN,0.742205,NaN,NaN
UNI3-USD,NaN,NaN,NaN,NaN,0.745021,NaN,NaN,NaN,NaN,0.462157,NaN,NaN,0.637369
LINK-USD,NaN,NaN,NaN,NaN,0.753549,NaN,NaN,NaN,NaN,0.590771,NaN,NaN,0.691853
LTC-USD,NaN,NaN,NaN,NaN,0.772738,NaN,NaN,NaN,NaN,0.630978,NaN,NaN,0.711569
BCH-USD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.752062,NaN,NaN,NaN
ICP1-USD,0.062579,0.177361,0.210663,0.429913,NaN,0.462157,0.590771,0.630978,0.752062,NaN,NaN,0.222901,0.730340


## Calculate annualized returns / volatility / sharpe ratio

## Create Portfolio

In [ ]:
# Play around with the weights?

## Compare annualized returns / volatility / sharpe ratio of individual cryptos vs portfolio